In [ ]:
# Colab cell 1 — GPU check & installs
!nvidia-smi
# Installing required packages
!pip install -q torch torchvision tqdm einops

In [ ]:
# cell 2 — imports
import os
import math
import time
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from einops import rearrange
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


device: cuda


In [ ]:
# cell 3 — ViT implementation (simple, modular)
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=4, emb_dim=192, img_size=32):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        # Using conv to patchify
        self.proj = nn.Conv2d(in_channels, emb_dim, kernel_size=patch_size, stride=patch_size)
    def forward(self, x):
        # x: (B, C, H, W)
        x = self.proj(x)  # (B, emb_dim, H/ps, W/ps)
        b, e, h, w = x.shape
        x = x.flatten(2).transpose(1,2)  # (B, num_patches, emb_dim)
        return x

class MLP(nn.Module):
    def __init__(self, in_dim, mlp_dim, drop=0.):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, mlp_dim)
        self.fc2 = nn.Linear(mlp_dim, in_dim)
        self.act = nn.GELU()
        self.drop = nn.Dropout(drop)
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=True, attn_drop=0., proj_drop=0.):
        super().__init__()
        assert dim % num_heads == 0
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim*3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads)
        q, k, v = qkv.unbind(dim=2)  # each is (B, N, heads, head_dim)
        q = q.transpose(1,2)  # (B, heads, N, head_dim)
        k = k.transpose(1,2)
        v = v.transpose(1,2)
        attn = (q @ k.transpose(-2,-1)) * self.scale  # (B, heads, N, N)
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        out = (attn @ v).transpose(1,2).reshape(B, N, C)
        out = self.proj(out)
        out = self.proj_drop(out)
        return out

class TransformerEncoderBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_dim, drop=0., attn_drop=0., drop_path=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = MultiHeadSelfAttention(dim, num_heads=num_heads, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = nn.Identity() if drop_path == 0. else nn.Dropout(drop_path)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(dim, mlp_dim, drop=drop)
    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class SimpleViT(nn.Module):
    def __init__(self, *, img_size=32, patch_size=4, in_chans=3, num_classes=10,
                 emb_dim=192, depth=12, num_heads=3, mlp_ratio=4., drop_rate=0.0):
        super().__init__()
        self.patch_embed = PatchEmbedding(in_chans, patch_size, emb_dim, img_size)
        num_patches = self.patch_embed.num_patches
        # cls token
        self.cls_token = nn.Parameter(torch.zeros(1,1,emb_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches+1, emb_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)
        # transformer blocks
        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(dim=emb_dim, num_heads=num_heads, mlp_dim=int(emb_dim*mlp_ratio), drop=drop_rate)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(emb_dim)
        # head
        self.head = nn.Linear(emb_dim, num_classes)
        # init
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(self._init_weights)
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)
    def forward(self, x):
        B = x.size(0)
        x = self.patch_embed(x)  # (B, num_patches, emb)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        cls = x[:,0]
        out = self.head(cls)
        return out


In [ ]:
# cell 4 — data (CIFAR-10) and augmentation
batch_size = 128
img_size = 32

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])

train_ds = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_ds  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)


100%|██████████| 170M/170M [00:21<00:00, 8.09MB/s]


In [ ]:
#  cell 5 — training utils and metrics
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append((correct_k.mul_(100.0 / batch_size)).item())
        return res

def evaluate(model, loader):
    model.eval()
    total = 0
    correct = 0
    losses = 0.0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            losses += loss.item() * x.size(0)
            _, pred = out.max(1)
            correct += (pred == y).sum().item()
            total += x.size(0)
    return losses/total, 100.0*correct/total


In [ ]:
#  cell 6 — instantiate model, optimizer, scheduler
# Hyperparameters — tweaking these for best results
model = SimpleViT(img_size=img_size, patch_size=4, emb_dim=256, depth=10, num_heads=8, mlp_ratio=4., num_classes=10, drop_rate=0.1)
model = model.to(device)
# optionally wraping with DataParallel if multiple GPUs
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
epochs = 80
criterion = nn.CrossEntropyLoss()
# Cosine scheduler with warmup
def cosine_scheduler(optimizer, epochs, warmup_epochs=5, base_lr=3e-4, final_lr=1e-5):
    lrs = []
    for epoch in range(epochs):
        if epoch < warmup_epochs:
            lr = base_lr * (epoch+1) / warmup_epochs
        else:
            t = (epoch - warmup_epochs) / (epochs - warmup_epochs)
            lr = final_lr + 0.5 * (base_lr - final_lr) * (1 + math.cos(math.pi * t))
        lrs.append(lr)
    return lrs
lrs = cosine_scheduler(optimizer, epochs)


In [ ]:
#  cell 7 — training loop (with optional mixed precision)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
best_acc = 0.0
for epoch in range(epochs):
    model.train()
    # set lr
    for param_group in optimizer.param_groups:
        param_group['lr'] = lrs[epoch]
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    running_loss = 0.0
    for imgs, labels in loop:
        imgs = imgs.to(device); labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            outputs = model(imgs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * imgs.size(0)
        loop.set_postfix(loss=running_loss/((loop.n+1)*train_loader.batch_size))
    # evaluating
    val_loss, val_acc = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}: test_loss={val_loss:.4f}, test_acc={val_acc:.2f}%")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_vit_cifar10.pth")
print("Best test acc:", best_acc)


/tmp/ipython-input-3092873707.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
Epoch 1/80:   0%|          | 0/391 [00:00<?, ?it/s]/tmp/ipython-input-3092873707.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Epoch 1/80: 100%|██████████| 391/391 [00:45<00:00,  8.56it/s, loss=2.13]


Epoch 1: test_loss=1.8121, test_acc=33.13%


Epoch 2/80: 100%|██████████| 391/391 [00:45<00:00,  8.66it/s, loss=1.92]


Epoch 2: test_loss=1.6541, test_acc=38.42%


Epoch 3/80: 100%|██████████| 391/391 [00:44<00:00,  8.71it/s, loss=1.79]


...

Epoch 78: test_loss=0.4551, test_acc=85.51%


Epoch 79/80: 100%|██████████| 391/391 [00:43<00:00,  8.97it/s, loss=0.479]


Epoch 79: test_loss=0.4558, test_acc=85.64%


Epoch 80/80: 100%|██████████| 391/391 [00:45<00:00,  8.69it/s, loss=0.479]


Epoch 80: test_loss=0.4598, test_acc=85.39%
Best test acc: 85.64
